<h1>SQL-анализ образовательных курсов</h1><br>

<h3>Задание 2. SQL</h3>

<h6>2.1 Очень усердные ученики</h6>
<h6>2.1.1 Условие</h6>

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий.<br>
Каждое такое маленькое задание называется "горошиной".<br>
Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц <b>правильно</b> решил 20 горошин.

<h6>2.1.2 Задача</h6>

Дана таблица:

 <b>default.peas</b>:
   - st_id (int) - id ученика;
   - timest (timestamp) - время решения карточки;
   - correct (bool) - правильно ли решена горошина;
   - subject (text) - дисциплина, в которой находится горошина.

Необходимо написать <b><u>оптимальный запрос</b></u>, который даст информацию о количестве очень усердных студентов.<br>
NB! <i>Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.</i>

<h4> Библиотеки

In [2]:
# Подключим библиотеки
import pandahouse as ph
import pandas as pd

In [9]:
# Cоздадим словарь с подключением с нужными параметрами для подключения к базе CH

connection_default_1 = {'host': 'https://clickhouse.lab.karpov.courses',
                                              'database':'default',
                                              'user':'student', 
                                              'password':'dpo_python_2020'
                                             }

In [10]:
# Запишем запрос
query_1 = '''
                        SELECT
                            COUNT(st_id) AS count_id_more_20_true
                        FROM
                         (
                            SELECT
                                st_id,
                                sum(correct) AS count_correct_in_month
                            FROM 
                                default.peas
                            GROUP BY
                                st_id,
                                toStartOfMonth(timest) AS month
                            HAVING
                                count_correct_in_month >= 20
                        )
'''

# Запишем результат в переменную
count_id_more_20_true = ph.read_clickhouse(query = query_1, connection = connection_default_1)

print("Количество усердных учеников = {}.".format(count_id_more_20_true.count_id_more_20_true[0]))

Количество усердных учеников = 136.


<h5>2.1 Очень усердные ученики</h5>
<h6>2.2.1 Условие</h6>

Образовательная платформа предлагает пройти студентам курсы по модели <i>trial</i>: <br>
студент может решить бесплатно лишь 30 горошин в день. <br>
Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. <br>
Команда провела эксперимент, где был протестирован новый экран оплаты.

<h6>2.2.2 Задача</h6>

Даны 3 таблицы:

<b>default.peas</b>:
   - st_id (int) - id ученика;
   - timest (timestamp) - время решения карточки;
   - correct (bool) - правильно ли решена горошина;
   - subject (text) - дисциплина, в которой находится горошина.<br>
   
<b>default.studs</b>:
   - st_id (int) - id ученика;
   - test_grp (text) - метка ученика в данном эксперименте.
   
<b>default.final_project_check</b>:
   - st_id (int) - id ученика;
   - sale_time (timestamp) - время покупки;
   - money (int) - цена, по которой приобрели данный курс;
   - subject (text) - дисциплина, в которой находится горошина.<br>


Необходимо <b><u>в одном запросе</b></u> выгрузить следующую информацию о группах пользователей:
- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике.

ARPU считается относительно всех пользователей, попавших в группы.

<b>Активным</b> считается пользователь, за все время решивший <b>больше</b> 10 задач правильно в любых дисциплинах.

<b>Активным</b> по математике считается пользователь, за все время решивший <b>2 или больше</b> задач правильно по математике.

In [25]:
# SQL код с комментариями (импорт в Python не пропускает код с комментариями)

"""WITH 
     --x1 - число всех студентов
     all_studs AS
     (SELECT
            test_grp,
            COUNT(DISTINCT st_id) AS all_studs
     FROM default.studs
     GROUP BY
            test_grp
     ORDER BY
            test_grp ASC
     ), -- число всех студентов 
      
     --x2 - общее число студентов всех студентов, что купили курсы
     all_paying_studs AS
    (SELECT
        test_grp,
        COUNT(DISTINCT st_id) AS all_paying_studs
     FROM
        (
        SELECT  
            R.test_grp AS test_grp,
            L.st_id AS st_id
        FROM default.final_project_check AS L
        JOIN default.studs AS R
        ON L.st_id = R.st_id
        )
     GROUP BY
        test_grp
     ORDER BY
        test_grp ASC
     ), -- общее число студентов всех студентов, что купили курсы
     
     --x3 - общая сумма всех покупок
     sum_revenue AS
     (SELECT
        test_grp,
        SUM(money) AS sum_revenue
     FROM
        (
        SELECT  
           R.test_grp AS test_grp,
           L.st_id AS st_id,
           L.money AS money
        FROM default.final_project_check AS L
        LEFT JOIN default.studs AS R
        ON L.st_id = R.st_id
        )
     GROUP BY
        test_grp
     ORDER BY 
        test_grp ASC
     ), -- общая сумма всех покупок
     
     --x4 - число активных студентов
     all_active_studs AS
     (SELECT 
        test_grp,
        COUNT(DISTINCT st_id) AS all_active_studs
      FROM 
        (
        SELECT 
            A.st_id AS st_id,
            A.test_grp AS test_grp,
            B.count_correct AS count_correct
        FROM default.studs AS A
        ---
        RIGHT JOIN (
                    SELECT
                        st_id,
                        sum(correct) AS count_correct
                    FROM 
                        default.peas
                    GROUP BY
                        st_id
                    ) AS B
        ON A.st_id = B.st_id
        HAVING count_correct >= 10
        )
     GROUP BY
        test_grp
     ORDER BY
        test_grp ASC
     ), -- число активных студентов
     
     --x5 - число активных студентов, которые покупали курсы
     all_paying_active_studs AS
     (SELECT
        test_grp,
        COUNT(DISTINCT st_id) AS all_paying_active_studs
      FROM
        (
        SELECT 
            L.test_grp AS test_grp,
            L.st_id AS st_id,
            R.money AS money
        FROM 
            (
            SELECT 
                A.st_id AS st_id,
                A.test_grp AS test_grp,
                B.count_correct AS count_correct
            FROM default.studs AS A
            ---
            RIGHT JOIN (
                        SELECT
                            st_id,
                            sum(correct) AS count_correct
                        FROM 
                            default.peas
                        GROUP BY
                            st_id
                        ) AS B
            ON A.st_id = B.st_id
            HAVING count_correct >= 10
            ) AS L
            ---
        JOIN (
            SELECT  
                st_id,
                money
            FROM
                default.final_project_check
        ) AS R  
        ON L.st_id = R.st_id
     )
     GROUP BY
        test_grp
     ORDER BY 
        test_grp ASC
     ), -- число активных студентов, которые покупали курсы
    
     --x6 - число активных студентов по Math
     all_active_math_studs AS
     (SELECT
        test_grp,
        COUNT(DISTINCT st_id) AS all_active_math_studs
      FROM
        (
        SELECT  
            L.st_id AS st_id,
            R.test_grp AS test_grp,
            L.count_correct_math AS count_correct_math
        FROM
            (
            SELECT
                st_id,
                sum(correct) AS count_correct_math
            FROM 
                default.peas
            WHERE 
                subject == 'Math'
            GROUP BY
                st_id,
                subject
            HAVING 
                count_correct_math >= 2
                
            ) AS L
        LEFT JOIN default.studs AS R
        ON L.st_id = R.st_id
        )
     GROUP BY
        test_grp
     ORDER BY 
        test_grp ASC
     ), -- число активных студентов по Math
      
     --x7 - число студентов, купивших курс Math, из числа активных студентов по Math
     all_paying_math_active_math_studs AS
     (SELECT 
        R.test_grp AS test_grp,
        COUNT (st_id) AS all_paying_math_active_math_studs
      FROM 
           (
            SELECT
                active_math_studs.st_id AS st_id,
                active_math_studs.count_correct_math AS count_correct_math,
                purchases_math_studs.money AS money
            FROM
                (
                SELECT
                    st_id,
                    sum(correct) AS count_correct_math
                FROM 
                    default.peas AS active_math_studs
                WHERE 
                    subject == 'Math'
                GROUP BY
                    st_id,
                    subject
                HAVING 
                    count_correct_math >= 2
                ) AS active_math_studs
            JOIN (
                SELECT  
                    st_id,
                    money
                FROM
                    default.final_project_check
                WHERE subject == 'Math'
                 ) AS purchases_math_studs  
            ON active_math_studs.st_id = purchases_math_studs.st_id
           ) AS L
     LEFT JOIN default.studs AS R
     ON L.st_id = R.st_id
     GROUP BY
        test_grp
     ORDER BY 
        test_grp ASC
     ) -- число студентов, купивших курс Math, из числа активных студентов по Math
        
---------------------------------------------------------------------------------------------------------------------------     

SELECT
    test_grp,
    ROUND(sum_revenue / all_studs, 3) AS ARPU,                                                                   -- средняя выручка со всех студентов ????
    ROUND(sum_revenue / all_active_studs, 3) AS ARPAU,                                                           -- средняя выручка со всех активных студентов
    ROUND((all_paying_studs / all_studs) * 100, 3) AS CR_all_studs_in_purchases,                                         -- конверсия всех студентов в покупку
    ROUND((all_paying_active_studs / all_active_studs) * 100, 3) AS CR_all_active_studs_in_purchases,                    -- конверсия активных студентов в покупку
    ROUND((all_paying_math_active_math_studs / all_active_math_studs) * 100, 3) AS CR_all_active_math_studs_in_purchases -- конверсия активных студентов по Math в покупку курса по Math

FROM
    (
    SELECT
        T1.test_grp AS test_grp,                                                    -- группы
        T1.all_studs AS all_studs,                                                  -- число всех студентов
        T2.all_paying_studs AS all_paying_studs,                                    -- общее число студентов всех студентов, что купили курсы
        T3.sum_revenue AS sum_revenue,                                              -- общая сумма всех покупок
        T4.all_active_studs AS all_active_studs,                                    -- число активных студентов
        T5.all_paying_active_studs AS all_paying_active_studs,                      -- число активных студентов, которые покупали курсы
        T6.all_active_math_studs AS all_active_math_studs,                          -- число активных студентов по Math
        T7.all_paying_math_active_math_studs AS all_paying_math_active_math_studs   -- число студентов, купивших курс Math, из числа активных студентов по Math
    FROM all_studs AS T1     
    JOIN all_paying_studs AS T2                       
    ON T1.test_grp = T2.test_grp 
    JOIN sum_revenue AS T3                       
    ON T1.test_grp = T3.test_grp
    JOIN all_active_studs AS T4                  
    ON T1.test_grp = T4.test_grp
    JOIN all_paying_active_studs AS T5           
    ON T1.test_grp = T5.test_grp
    JOIN all_active_math_studs AS T6             
    ON T1.test_grp = T6.test_grp
    JOIN all_paying_math_active_math_studs AS T7 
    ON T1.test_grp = T7.test_grp
)
"""
"#"

'#'

In [24]:
# Код запроса без комментариев (импорт в Python не пропускает код с комментариями)

query_metrics = """WITH 
     all_studs AS
     (SELECT
            test_grp,
            COUNT(DISTINCT st_id) AS all_studs
     FROM default.studs
     GROUP BY
            test_grp
     ORDER BY
            test_grp ASC
     ),
      

     all_paying_studs AS
    (SELECT
        test_grp,
        COUNT(DISTINCT st_id) AS all_paying_studs
     FROM
        (
        SELECT  
            R.test_grp AS test_grp,
            L.st_id AS st_id
        FROM default.final_project_check AS L
        JOIN default.studs AS R
        ON L.st_id = R.st_id
        )
     GROUP BY
        test_grp
     ORDER BY
        test_grp ASC
     ),
     

     sum_revenue AS
     (SELECT
        test_grp,
        SUM(money) AS sum_revenue
     FROM
        (
        SELECT  
           R.test_grp AS test_grp,
           L.st_id AS st_id,
           L.money AS money
        FROM default.final_project_check AS L
        LEFT JOIN default.studs AS R
        ON L.st_id = R.st_id
        )
     GROUP BY
        test_grp
     ORDER BY 
        test_grp ASC
     ),
     

     all_active_studs AS
     (SELECT 
        test_grp,
        COUNT(DISTINCT st_id) AS all_active_studs
      FROM 
        (
        SELECT 
            A.st_id AS st_id,
            A.test_grp AS test_grp,
            B.count_correct AS count_correct
        FROM default.studs AS A
        RIGHT JOIN (
                    SELECT
                        st_id,
                        sum(correct) AS count_correct
                    FROM 
                        default.peas
                    GROUP BY
                        st_id
                    ) AS B
        ON A.st_id = B.st_id
        HAVING count_correct >= 10
        )
     GROUP BY
        test_grp
     ORDER BY
        test_grp ASC
     ),
     

     all_paying_active_studs AS
     (SELECT
        test_grp,
        COUNT(DISTINCT st_id) AS all_paying_active_studs
      FROM
        (
        SELECT 
            L.test_grp AS test_grp,
            L.st_id AS st_id,
            R.money AS money
        FROM 
            (
            SELECT 
                A.st_id AS st_id,
                A.test_grp AS test_grp,
                B.count_correct AS count_correct
            FROM default.studs AS A
            RIGHT JOIN (
                        SELECT
                            st_id,
                            sum(correct) AS count_correct
                        FROM 
                            default.peas
                        GROUP BY
                            st_id
                        ) AS B
            ON A.st_id = B.st_id
            HAVING count_correct >= 10
            ) AS L
        JOIN (
            SELECT  
                st_id,
                money
            FROM
                default.final_project_check
        ) AS R  
        ON L.st_id = R.st_id
     )
     GROUP BY
        test_grp
     ORDER BY 
        test_grp ASC
     ),
    

     all_active_math_studs AS
     (SELECT
        test_grp,
        COUNT(DISTINCT st_id) AS all_active_math_studs
      FROM
        (
        SELECT  
            L.st_id AS st_id,
            R.test_grp AS test_grp,
            L.count_correct_math AS count_correct_math
        FROM
            (
            SELECT
                st_id,
                sum(correct) AS count_correct_math
            FROM 
                default.peas
            WHERE 
                subject == 'Math'
            GROUP BY
                st_id,
                subject
            HAVING 
                count_correct_math >= 2
                
            ) AS L
        LEFT JOIN default.studs AS R
        ON L.st_id = R.st_id
        )
     GROUP BY
        test_grp
     ORDER BY 
        test_grp ASC
     ),
      
 
     all_paying_math_active_math_studs AS
     (SELECT 
        R.test_grp AS test_grp,
        COUNT (st_id) AS all_paying_math_active_math_studs
      FROM 
           (
            SELECT
                active_math_studs.st_id AS st_id,
                active_math_studs.count_correct_math AS count_correct_math,
                purchases_math_studs.money AS money
            FROM
                (
                SELECT
                    st_id,
                    sum(correct) AS count_correct_math
                FROM 
                    default.peas AS active_math_studs
                WHERE 
                    subject == 'Math'
                GROUP BY
                    st_id,
                    subject
                HAVING 
                    count_correct_math >= 2
                ) AS active_math_studs
            JOIN (
                SELECT  
                    st_id,
                    money
                FROM
                    default.final_project_check
                WHERE subject == 'Math'
                 ) AS purchases_math_studs  
            ON active_math_studs.st_id = purchases_math_studs.st_id
           ) AS L
     LEFT JOIN default.studs AS R
     ON L.st_id = R.st_id
     GROUP BY
        test_grp
     ORDER BY 
        test_grp ASC
     )
        
---------------------------------------------------------------------------------------------------------------------------     

SELECT
    test_grp,
    ROUND(sum_revenue / all_studs, 3) AS ARPU,                                                                   
    ROUND(sum_revenue / all_active_studs, 3) AS ARPAU,                                                           
    ROUND((all_paying_studs / all_studs) * 100, 3) AS CR_all_studs_in_purchases,                                         
    ROUND((all_paying_active_studs / all_active_studs) * 100, 3) AS CR_all_active_studs_in_purchases,                    
    ROUND((all_paying_math_active_math_studs / all_active_math_studs) * 100, 3) AS CR_all_active_math_studs_in_purchases 

FROM
    (
    SELECT
        T1.test_grp AS test_grp,                                                    
        T1.all_studs AS all_studs,                                                 
        T2.all_paying_studs AS all_paying_studs,                                   
        T3.sum_revenue AS sum_revenue,                                           
        T4.all_active_studs AS all_active_studs,                               
        T5.all_paying_active_studs AS all_paying_active_studs,                     
        T6.all_active_math_studs AS all_active_math_studs,                      
        T7.all_paying_math_active_math_studs AS all_paying_math_active_math_studs
    FROM all_studs AS T1     
    JOIN all_paying_studs AS T2                       
    ON T1.test_grp = T2.test_grp 
    JOIN sum_revenue AS T3                       
    ON T1.test_grp = T3.test_grp
    JOIN all_active_studs AS T4                  
    ON T1.test_grp = T4.test_grp
    JOIN all_paying_active_studs AS T5           
    ON T1.test_grp = T5.test_grp
    JOIN all_active_math_studs AS T6             
    ON T1.test_grp = T6.test_grp
    JOIN all_paying_math_active_math_studs AS T7 
    ON T1.test_grp = T7.test_grp
)
"""

In [18]:
connection_default_2 = {'host': 'https://clickhouse.lab.karpov.courses',
                        'database':'default',
                        'user':'student-rw', 
                        'password':'656e2b0c9c'
                        }

connection_default_1 = {'host': 'https://clickhouse.lab.karpov.courses',
                        'database':'default',
                        'user':'student', 
                        'password':'dpo_python_2020'
                        }

In [22]:
metrics = ph.read_clickhouse(query = query_metrics, connection = connection_default_1)
metrics

,test_grp,ARPU,ARPAU,CR_all_studs_in_purchases,CR_all_active_studs_in_purchases,CR_all_active_math_studs_in_purchases
0,control,4540.984,10492.424,4.918,10.606,6.122
1,pilot,11508.475,33613.861,10.847,24.752,9.524
